In [40]:
import os
from datasets import load_dataset
import pandas as pd
from huggingface_hub import login

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import bitsandbytes as bnb
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training)

from typing import Union
from dotenv import load_dotenv

!pip install -q -U bitsandbytes
!pip install datasets -U
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install wandb
!pip install pandas
!pip install python-dotenv

In [41]:
load_dotenv('.env')
api_key = os.getenv("API_KEY")

In [42]:
my_hf_key = api_key
login(my_hf_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/user1/.cache/huggingface/token
Login successful


In [3]:
model_path = 'meta-llama/Llama-3.1-8B-Instruct'
data_path = 'DopeorNope/Ko-Optimize_Dataset'

In [4]:
data = load_dataset(data_path)

In [5]:
df = pd.DataFrame(data['train'])

In [6]:
df.head()

,input,instruction,output
0,,"∫[0,t] (GCP × SM × TA) dt = SA\n\n여기서\n∫ = 적분 ...","예, 주어진 공식을 다음과 같이 수정할 수 있습니다:\n\n∫[0,T] [(GCP ..."
1,귀하는 사람들이 정보를 찾도록 도와주는 AI 어시스턴트입니다. 사용자가 질문을 합니...,정답 -> 문장 A\n질문 -> 다음 두 문장 중 상식에 어긋나는 것은 어느 것입니...,질문 -> 다음 두 문장 중 상식에 어긋나는 것은 어느 것입니까?\n옵션:\n- 문...
2,,이산화탄소 배출을 줄이는 5가지 방법을 나열하세요.,"1. 재생 에너지원 사용: 태양열, 풍력, 수력, 지열 등 이산화탄소를 배출하지 않..."
3,귀하는 사람들이 정보를 찾도록 도와주는 AI 어시스턴트입니다. 사용자가 질문을 합니...,"많은 정치인들이 소농에 대해 이야기하는 것을 좋아하지만, 실제로 거의 모든 농장은 ...",서서히 해봅시다: 노화된 록스타의 손이 협조하지 않는 것은 건강 문제를 나타냅니다....
4,귀하는 항상 설명을 제공하는 도움이 되는 조수입니다. 5살짜리 아이에게 대답한다고 ...,다음 리뷰의 감상을 알려주세요: 솔직히 저는 사춘기 욕구 때문에 이 영화를 봤어요....,이 리뷰에는 긍정적인 감정이 담겨 있습니다. 영화를 보고 유머와 연기를 즐기며 즐거...


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [11]:
# 토크나이저 세팅 : QLoRA시 pad 토큰을 eos로 설정하기
bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

tokenizer.add_special_tokens({"pad_token" : "<|reserved_special_token_0>"})

tokenizer.pad_token_id = eos
tokenizer.padding_side = 'right'

In [13]:
cut_off_len = 4098
val_size = 0.005
train_on_inputs = False
add_eos_token = False

In [20]:
template = {
    "prompt_input": "아래는 문제를 설명하는 지시사항과, 구체적인 답변을 방식을 요구하는 입력이 함께 있는 문장입니다. 이 요청에 대해 적절하게 답변해주세요.\n###입력:{input}\n###지시사항:{instruction}\n###답변:",
    "prompt_no_input": "아래는 문제를 설명하는 지시사항입니다. 이 요청에 대해 적절하게 답변해주세요.\n###지시사항:{instruction}\n###답변"
}

In [27]:
def generate_prompt(
    instruction: str,
    input: Union[None, str] = None,
    label: Union[None, str] = None,
    verbose: bool = False
) -> str:
    if input:
        res = template["prompt_input"].format(instruction=instruction, input=input)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)
    if label:
        res = f"{res}{label}"
    if verbose:
        print(res)

    return res

In [28]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(prompt, truncation=True, max_length=cut_off_len, padding=False, return_tensors=None,)
    if (result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cut_off_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    result["labels"] = result["input_ids"].copy()
    return result

In [31]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"]
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = generate_prompt(data_point["instruction"], data_point["input"])
        tokenized_user_prompt = tokenize(user_prompt, add_eos_token=add_eos_token)
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [-100] * user_prompt_len + tokenized_full_prompt["labels"][user_prompt_len:]
    return tokenized_full_prompt

In [32]:
if val_size > 0:
    train_val = data["train"].train_test_split(test_size = val_size, shuffle=True, seed=42)
    train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
    val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt))
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

Map:   0%|          | 0/9950 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

#### Model 

In [35]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16
)

In [36]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quantization_config,
    torch_dtype = torch.bfloat16,
    device_map = {"": 0}
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [43]:
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r = 16,
    lora_alpha = 16
    target_modules = ['q_proj','k_proj', 'v_proj', 'o_proj']
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
    
)